![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
import pandas as pd

import os

import time

import numpy as np

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "../UCI HAR Dataset/UCI HAR Dataset/features.txt"
filename_labels = "../UCI HAR Dataset/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "../UCI HAR Dataset/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "../UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "../UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "../UCI HAR Dataset/UCI HAR Dataset/test/subject_test.txt"
filename_xtest = "../UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "../UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt"

# Load the features
features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
features_list = features['nome_var'].tolist()  # Convert column to list

# Load labels and train/test data
labels = pd.read_csv(filename_labels, sep=r'\s+', header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, sep=r'\s+', header=None, names=features_list)
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(filename_xtest, sep=r'\s+', header=None, names=features_list)
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [2]:
%%time
clf = DecisionTreeClassifier(ccp_alpha = 0.001 , random_state= 21).fit(X_train , y_train)

CPU times: total: 5.11 s
Wall time: 5.55 s


In [3]:
print('Acurácia na base de treino {:.1f}'.format(clf.score(X_train , y_train) * 100))
print('Acurácia na base de teste {:.1f}'.format(clf.score(X_test , y_test) * 100))

Acurácia na base de treino 97.6
Acurácia na base de teste 88.0


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [4]:
%%time
prcomp = PCA(n_components=1).fit(X_train)
pc_train = prcomp.transform(X_train)
pc_test = prcomp.transform(X_test)

CPU times: total: 219 ms
Wall time: 192 ms


In [5]:
clf.fit(pc_train , y_train)
test_acc = clf.score(pc_test , y_test)
train_acc = clf.score(pc_train , y_train)

In [6]:
print('Acurácia na base de treino {:.1f}'.format(train_acc * 100))
print('Acurácia na base de teste {:.1f}'.format(test_acc * 100))

Acurácia na base de treino 50.0
Acurácia na base de teste 45.7


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [19]:
path = DecisionTreeClassifier(random_state= 21).cost_complexity_pruning_path(pc_train , y_train)
ccp_alphas , impurities = path.ccp_alphas , path.impurities

ccp_alphas = np.unique(ccp_alphas[ccp_alphas>=0])

clfs = []

for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state= 21 , ccp_alpha= ccp_alpha).fit(pc_train , y_train)
    clfs.append(clf)

In [26]:
%%time
n = [1 , 2 , 5 , 10 , 50]
train_acc = []
test_acc = []
time_process = []

for n_comp in n:
    start = time.time()
    prcomp = PCA(n_components=n_comp).fit(X_train)
    pc_train = prcomp.transform(X_train)
    pc_test = prcomp.transform(X_test)
    
    clf = DecisionTreeClassifier(ccp_alpha = 0.001 , random_state= 21).fit(pc_train , y_train) 
    train_acc.append(clf.score(pc_train , y_train) * 100)
    
    clf = clf.fit(pc_test , y_test)
    test_acc.append(clf.score(pc_test , y_test) * 100)
    end = (time.time() - start)
    time_process.append(end)

CPU times: total: 2.36 s
Wall time: 2.17 s


In [35]:
print('Acurácia na base de treino {:.1f}'.format(max(train_acc)))
print('Acurácia na base de teste {:.1f}'.format(max(test_acc)))
print('O tempo de processamento na árvore de {} componentes foi de {} segundos'.format((max(n)) , round(max(time_process) , 2)))

Acurácia na base de treino 91.9
Acurácia na base de teste 97.3
O tempo de processamento na árvore de 50 componentes foi de 1.08 segundos


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

A árvore com melhor acurácia 91.9% na base de treino e 97.3% na base de testes, foi a de 50 variáveis e o tempo de processamento foi de 1.08 segundos.